In [1]:
import tensorflow as tf
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from helper_functions import walk_through_dir

BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

def should_ignore(path, ignored_paths):
    return any(ignored_path in path for ignored_path in ignored_paths)

ignored_paths = ['LOONEY BIRDS', 'TURKEY', 'PENGUIN', 'OWL', 'EAGLE', 'EMU',
                 'VULTURE', 'CHICKEN', 'GOOSE', 'OSTRICH', 'CONDOR', 'PELICAN',
                'CASSOWARY', 'FLAMINGO', "CRANE", "SWAN", "PEACOCK", "IBIS",
                 "STORK", "PHEASANT", "FALCON", "OSPREY", "HAWK", "CURASSOW",
                 "VULTURINE", "HEN", "HUMMINGBIRD", "SWALLOW", "WARBLER", "FLYCATCHER",
                 "KINGFISHER", "HERON", "EGRET", "KINGLET", "SHRIKE", "WREN"]


dataset = "C:/colab/data/birds/train"
walk_through_dir(dataset);
image_dir = Path(dataset)
filepaths = [str(path) for path in image_dir.glob('**/*') if path.is_file() and not should_ignore(str(path), ignored_paths)]
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
train_df = pd.concat([filepaths, labels], axis=1)

dataset = "C:/colab/data/birds/test"
walk_through_dir(dataset);
image_dir = Path(dataset)
filepaths = [str(path) for path in image_dir.glob('**/*') if path.is_file() and not should_ignore(str(path), ignored_paths)]
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
test_df = pd.concat([filepaths, labels], axis=1)

dataset = "C:/colab/data/birds/valid"
walk_through_dir(dataset);
image_dir = Path(dataset)
filepaths = [str(path) for path in image_dir.glob('**/*') if path.is_file() and not should_ignore(str(path), ignored_paths)]
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
valid_df = pd.concat([filepaths, labels], axis=1)

train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
)

val_images = train_generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [4]:
print("Starting model testing")
model_path = 'C:/colab/data/birds/bird_classifier - 88.tflite'
tflite_interpreter = tf.lite.Interpreter(model_path=model_path)
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

tflite_interpreter.resize_tensor_input(input_details[0]['index'], (32, 224, 224, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (32, 5))
tflite_interpreter.allocate_tensors()

model_size_bytes = os.path.getsize(model_path)

  
print(f"Model Size: {model_size_bytes} bytes")

input_batch, _ = next(test_images)


results = []

for i in range(len(test_images)):
    batch = test_images[i]
    input_batch, labels = batch
    image_input_batch = tf.cast(input_batch, tf.int8)

    input_scale, input_zero_point = input_details[0]["quantization"]
    image_input_batch = tf.cast(image_input_batch, tf.float32)
    image_input_batch = (image_input_batch / input_scale) + input_zero_point
    image_input_batch = tf.cast(image_input_batch, tf.int8)

    if len(image_input_batch) == 32:
        tflite_interpreter.set_tensor(input_details[0]['index'], image_input_batch)
        tflite_interpreter.invoke()
        tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    
        tflite_predicted_ids = np.argmax(tflite_model_predictions, axis=-1)
        tflite_label_id = np.argmax(labels, axis=-1)
    
        correct_predictions = np.sum(tflite_predicted_ids == tflite_label_id)
        total_predictions = len(tflite_predicted_ids)
        accuracy_percentage = (correct_predictions / total_predictions) * 100
    
        results.append(accuracy_percentage)
        average_accuracy = sum(results) / len(results)
        print("\nIteration ", i, ": ", accuracy_percentage, " - Running Average: ", average_accuracy)

print("\nEnd Average: ", average_accuracy)

Starting model testing
Model Size: 216256 bytes

Iteration  0 :  78.125  - Running Average:  78.125

Iteration  1 :  87.5  - Running Average:  82.8125

Iteration  2 :  78.125  - Running Average:  81.25

Iteration  3 :  90.625  - Running Average:  83.59375

Iteration  4 :  87.5  - Running Average:  84.375

Iteration  5 :  75.0  - Running Average:  82.8125

Iteration  6 :  90.625  - Running Average:  83.92857142857143

Iteration  7 :  93.75  - Running Average:  85.15625

Iteration  8 :  84.375  - Running Average:  85.06944444444444

Iteration  9 :  90.625  - Running Average:  85.625

Iteration  10 :  87.5  - Running Average:  85.79545454545455

Iteration  11 :  84.375  - Running Average:  85.67708333333333

Iteration  12 :  84.375  - Running Average:  85.57692307692308

Iteration  13 :  65.625  - Running Average:  84.15178571428571

Iteration  14 :  75.0  - Running Average:  83.54166666666667

Iteration  15 :  93.75  - Running Average:  84.1796875

Iteration  16 :  100.0  - Running Avera

In [4]:
model_h_path = "C:/colab/data/birds/model.h"

with open(model_path, "rb") as tflite_file:
    model_data = tflite_file.read()
    model_hex_array = ', '.join(f'0x{byte:02X}' for byte in model_data)


with open(model_h_path, "w") as model_h_file:
    model_h_file.write(f"const unsigned char model[] __attribute__((aligned(4))) = {{\n    {model_hex_array}\n}};\n")


model_h_size = os.path.getsize(model_h_path)
print(f"Header file, model.h, is {model_h_size:,} bytes.")


Header file, model.h, is 1,297,605 bytes.


In [7]:
import csv

def generate_c_function(csv_file_path, output_file_path):
    # Load labels from CSV
    labels = {}
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            labels[int(row['ClassIndex'])] = row['Label']

    # Generate C function
    c_function = "#include <stdio.h>\n\n"
    c_function += "const char* getBirdLabel(int classIndex) {\n"
    c_function += "    switch(classIndex) {\n"

    for class_index, label in labels.items():
        c_function += f"        case {class_index}: return \"{label}\";\n"

    c_function += "        default: return \"Unknown\";\n"
    c_function += "    }\n"
    c_function += "}\n"

    # Save C function to file
    with open(output_file_path, 'w') as c_file:
        c_file.write(c_function)

    print("C function generated and saved to:", output_file_path)

# Example usage
generate_c_function('C:/colab/data/birds/labels.csv', 'C:/colab/data/birds/labels.h')


C function generated and saved to: C:/colab/data/birds/labels.h
